In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 1927, done.
remote: Counting objects: 100% (1927/1927), done.
remote: Compressing objects: 100% (1684/1684), done.
remote: Total 1927 (delta 444), reused 769 (delta 225), pack-reused 0
Receiving objects: 100% (1927/1927), 30.09 MiB | 34.62 MiB/s, done.
Resolving deltas: 100% (444/444), done.


In [3]:
!pip install pycocotools

In [4]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1550866 sha256=40d4d46de0bff9b9e4ca903299d8fe5a2ce7d01e7b3e8f2d8e314182a5163b6e
  Stored in directory: /tmp/pip-ephem-wheel-cache-dodqo511/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=48a66da7b7e88f4e2d41bf17846ac3889b6b635f4b1d25a91ddb262a74614542
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=28279578f6bf32734ecff33187c16eee5539ef60fda8adc0d909cae32daf57d2
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=b31406bc1e87dd4fc7ef049a22592bc0eb1de379adfd9d635d3d

ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: apache-beam 2.23.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.10.0 which is incompatible.


In [5]:
!git clone https://github.com/tarun-bisht/tensorflow-object-detection.git
os.chdir("tensorflow-object-detection")

Cloning into 'tensorflow-object-detection'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 41 (delta 8), reused 36 (delta 6), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [6]:
# Download a pretrained model from tensorflow model zoo
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz
!rm efficientdet_d0_coco17_tpu-32.tar.gz
!mv efficientdet_d0_coco17_tpu-32 data/models

--2020-08-23 16:59:38--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.195.128, 2607:f8b0:400e:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M   146MB/s    in 0.2s    

2020-08-23 16:59:38 (146 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]



In [8]:
# load data from google drive for training
!unzip -q ../../gdrive/My\ Drive/detection_data_train.zip -d  ./data/images/train
!unzip -q ../../gdrive/My\ Drive/detection_data_test.zip -d  ./data/images/test

In [9]:
!python create_dataset.py --labels data/labels/theft.pbtxt

2020-08-23 17:04:06.462107: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Successfully converted test annotations to csv at data/csv/test_labels.csv
Successfully created the TFRecords: data/tfrecords/test.record
Successfully converted train annotations to csv at data/csv/train_labels.csv
Successfully created the TFRecords: data/tfrecords/train.record


In [10]:
!python train.py --model_dir=../../gdrive/My\ Drive/models/efficientdet_d0_coco17_tpu-32-theft --pipeline_config_path=data/models/efficientdet_d0_coco17_tpu-32/pipeline.config

2020-08-23 17:11:54.440449: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-23 17:11:57.062413: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-23 17:11:57.120967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-23 17:11:57.121589: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-08-23 17:11:57.121645: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-23 17:11:57.351004: I tensorflow/stream_executor/platform/default

In [12]:
!python export_model.py --input_type image_tensor --pipeline_config_path data/models/efficientdet_d0_coco17_tpu-32/pipeline.config --trained_checkpoint_dir ../../gdrive/My\ Drive/models/efficientdet_d0_coco17_tpu-32-theft --output_directory data/models/efficientdet_d0_coco17_tpu-32-theft

2020-08-23 17:36:56.893992: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-23 17:36:58.862249: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-23 17:36:58.899543: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-23 17:36:58.900097: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-08-23 17:36:58.900133: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-23 17:36:58.902926: I tensorflow/stream_executor/platform/default

In [13]:
!zip efficientdet_d0_coco17_tpu-32-theft.zip -r data/models/efficientdet_d0_coco17_tpu-32-theft

  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/ (stored 0%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/saved_model/ (stored 0%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/saved_model/variables/ (stored 0%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/saved_model/variables/variables.index (deflated 77%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/saved_model/variables/variables.data-00000-of-00001 (deflated 36%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/saved_model/saved_model.pb (deflated 93%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/saved_model/assets/ (stored 0%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/pipeline.config (deflated 68%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/checkpoint/ (stored 0%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/checkpoint/checkpoint (deflated 41%)
  adding: data/models/efficientdet_d0_coco17_tpu-32-theft/che

In [14]:
!mv efficientdet_d0_coco17_tpu-32-theft.zip ../../gdrive/My\ Drive/models/efficientdet_d0_coco17_tpu-32-theft.zip